In [10]:
import numpy as np

class Simplex:
    def __init__(self, c, A, b):
        if (np.less_equal(np.zeros(len(b)), b).all):
            self.c = c
            self.A = A
            self.b = b
            #print("tres")
            
        else:
            raise ValueError("Origin is not feasible")
            
    
    def index(self):
        A = self.A
        m, n = A.shape
        a = np.arange(n, n + m)
        b = np.arange(0, n)
        L = np.hstack([a, b])
        L = np.ndarray.tolist(L)
        return L

    def get_tableau(self):
        A = self.A
        b = self.b
        c = self.c
        m, n = A.shape
        c_bar = np.concatenate([c, np.zeros(m)])
        c_bar = -1 * c_bar
        T1 = np.concatenate([np.array([0]), c_bar, np.array([1])])
        A_bar = np.concatenate([A, np.eye(m)], axis = 1)
        T2 = np.hstack([b.reshape(len(b), 1), A_bar, np.zeros((m, 1))])
        T = np.vstack([T1, T2])
        return T
    
    
    def pivot(self, T):
        p, q = T.shape
        T1 = T[0, 1:]
        column = np.where(T1 < 0)[0][0] + 1
        if any(T[:, column] > 0)== False:
            raise ValueError
        else:
            loc = np.where(T[:, column] > 0)[0]
            n = len(loc)
            mat = np.zeros(n) 
            for i in range(n):
                mat[i] = float(T[loc[i], 0]/T[loc[i], column])
            m = np.where(mat==min(mat))[0][0]
            row = loc[m]+1

        return row , column+1 



        

    def op(self, T, L):
        T1 = np.copy(T)
        m, n = self.pivot(T)
        a, b = m - 2, L.index(n - 2) 
        p, q = T.shape
        m = m - 1
        n = n - 1
        if T[m, n] != 0:
             T1[m,:] = T[m, :]/float(T[m, n])
        for i in range(p):
            if i == m or T[i, n] == 0:
                T1[i, :] = T1[i, :]
            else:
                T1[i, :] = T1[i, :] - float((T1[i, n]/T1[m, n])) * T1[m, :]

        else:
             T1[m,:] = T[m, :]

        L[a], L[b] = L[b], L[a]
        return T1, L

    def solve(self, T, L):
        tol = -1e-8
        while any(T[0, 1:] < tol) == True:
            T_new, L_new = self.op(T, L)
            T = np.copy(T_new)
            L = L_new
        value = np.ndarray.tolist(T[1:,0])
        n = len(value)
        D1 = dict(zip(L[:n], [float(str(round(i, 2))) for i in value]))
        D2 = dict(zip((L[n:]), np.zeros(len(L) - n)))
        return float(T[0,0]), D1, D2
    
    

In [11]:
prob = np.load('productMix.npz')
A = prob['A']
p = prob['p']
m = prob['m']
d = prob['d']

A = np.vstack([A, np.eye(4)])
b = np.hstack([m, d])

s1 = Simplex(p, A, b)
L = s1.index()
T = s1.get_tableau()
s1.solve(T, L)
#s1.solve()

#T = tableau(p, A, b)


(7453.596491228071,
 {0: 10.0, 1: 1.11, 2: 12.0, 3: 0.11, 6: 0.97, 8: 13.81, 10: 8.21},
 {4: 0.0, 5: 0.0, 7: 0.0, 9: 0.0})